In [ ]:
# from sklearn.model_selection import train_test_split
# import numpy as np
# import os
# import tensorflow as tf

# data_path = os.path.join(os.getcwd(), 'data')
# X = np.load(os.path.join(data_path, 'X_69_200.npy'), allow_pickle=True)
# Y = np.load(os.path.join(data_path, 'Y_69_200.npy'), allow_pickle=True)

# freq_bins = X[0].shape[0]
# min_spec_len = min([x.shape[1] for x in X])
# data_size = X.shape[0]
# print(data_size, freq_bins, min_spec_len)

# # X = np.reshape(X, newshape=(data_size, nfft_bins,None ))
# # for i in range(data_size):
#     # x,y = X[i] , np.copy(Y[i])
#     # X[i], Y[i] = x.resize((freq_bins, min_spec_len)), y.resize((freq_bins, min_spec_len))
# X = [tf.stack(x[:,:min_spec_len]) for x in X]
# Y = [tf.stack(y[:,:min_spec_len]) for y in Y]

# splits = train_test_split(X, Y, test_size=0.1, random_state=69)
# X_train, X_test, y_train, y_test = [tf.convert_to_tensor(s) for s in splits]
# for item in [X_train, X_test, y_train, y_test]:
#     print(item.shape, item.dtype, type(item))

In [ ]:
import os
from pydub import AudioSegment
from scipy.signal import stft
from keras.preprocessing.sequence import TimeseriesGenerator

# Set the directory containing the clean and distorted speech files
clean_speech_dir = 'clean_speech_files'
distorted_speech_dir = 'distorted_speech_files'

# List all the files in the clean and distorted speech directories
clean_speech_files = os.listdir(clean_speech_dir)
distorted_speech_files = os.listdir(distorted_speech_dir)

# Loop through the clean and distorted speech files
for clean_speech_file, distorted_speech_file in zip(clean_speech_files, distorted_speech_files):
    # Read and load the clean and distorted audio files
    clean_speech = AudioSegment.from_file(os.path.join(clean_speech_dir, clean_speech_file))
    distorted_speech = AudioSegment.from_file(os.path.join(distorted_speech_dir, distorted_speech_file))
    
    # Convert the clean and distorted audio to a sequence of audio samples
    clean_speech_samples = np.array(clean_speech.get_array_of_samples())
    distorted_speech_samples = np.array(distorted_speech.get_array_of_samples())
    
    # Apply the STFT to the audio samples to get the spectrogram frames
    clean_speech_frames = stft(clean_speech_samples)
    distorted_speech_frames = stft(distorted_speech_samples)
    
    # Split the spectrogram frames into overlapping sequences of fixed length
    input_sequences = split_spectrogram_frames(distorted_speech_frames, sequence_length=128, overlap=64)
    target_sequences = split_spectrogram_frames(clean_speech_frames, sequence_length=128, overlap=64)
    
    # Create a data generator using the Timeseries
